# 0.1 WavesExecuter
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.

Althoght `WavesExecuter` has provided you to run multiple circuits at once by `.measure`.
But you can not set shots for each circuit for they run by same experiment.


### a. Import the instances


In [1]:
from qurry import WavesExecuter

experiment_workflow = WavesExecuter()

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


circuits_dict = {}

for i in range(2, 13, 2):
    circuits_dict[f"trivial_paramagnet_{i}_ms"] = TrivialParamagnet(i)
    circuits_dict[f"ghz_{i}_ms"] = GHZ(i)
    circuits_dict[f"neel_{i}_ms"] = make_neel_circuit(i)

for name, circuit in circuits_dict.items():
    circuit.measure_all()
    experiment_workflow.add(circuit, name)

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrium.wavesqurry.arguments import WavesExecuterMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class WavesExecuterMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    waves: Optional[list[Union[QuantumCircuit, Hashable]]]
    """The list of circuits to be measured."""
```


In [4]:
config_list: list[WavesExecuterMeasureArgs] = [
    {
        "shots": 1024,
        "waves": [f"{wave_names}_{i}_ms" for _ in range(10) for i in range(2, 13, 2)],
        "tags": (wave_names, f"size_{i}"),
    }
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

3


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_workflow.multiOutput(
    config_list,
    summoner_name="qurrium.waves_executer",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| qurrium.waves_executer.001 is repeat location.
| qurrium.waves_executer.002 is repeat location.
| qurrium.waves_executer.003 is repeat location.
| qurrium.waves_executer.004 is repeat location.
| qurrium.waves_executer.005 is repeat location.
| qurrium.waves_executer.006 is repeat location.
| qurrium.waves_executer.007 is repeat location.
| qurrium.waves_executer.008 is repeat location.
| qurrium.waves_executer.009 is repeat location.
| Write "qurrium.waves_executer.010", at location "qurrium.waves_executer.010"


| 0/3   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/3   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 6091db19-b385-4b74-b18c-3d551dd8e6b1


| 0/2 - Exporting experiments... - 00:00 < ?

| 0/3 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrium.waves_executer.010/qurryinfo.json...
| Exporting qurrium.waves_executer.010/qurryinfo.json done.


'6091db19-b385-4b74-b18c-3d551dd8e6b1'

In [6]:
experiment_workflow.multimanagers[multi_exps1]

<MultiManager(id="6091db19-b385-4b74-b18c-3d551dd8e6b1",
  name="qurrium.waves_executer.010",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-06-13 21:35:40',},
  exps_num=3)>

### d. Take all `counts`

In [7]:
for exp_id, exp_instance in experiment_workflow.multimanagers[multi_exps1].exps.items():
    print(f"| {exp_id}:", exp_instance.afterwards.counts)
    print("-" * 20)

| c1d573f6-8b69-4d4b-b8b7-c64d95649c80: [{'11': 277, '01': 246, '00': 244, '10': 257}, {'1101': 71, '1111': 56, '1110': 61, '0100': 55, '1011': 64, '0011': 72, '0111': 63, '0001': 62, '1100': 68, '0101': 54, '1001': 62, '0000': 68, '0010': 69, '1000': 70, '0110': 65, '1010': 64}, {'011111': 12, '010111': 13, '001111': 18, '011011': 11, '010000': 12, '100111': 14, '000100': 26, '110100': 19, '100100': 12, '101110': 14, '010100': 24, '101101': 21, '001110': 17, '100110': 22, '100000': 15, '000011': 13, '100001': 13, '010101': 18, '100010': 14, '111000': 9, '101010': 15, '101100': 22, '110111': 18, '010001': 13, '000110': 9, '011101': 14, '011010': 16, '000001': 19, '111100': 13, '010110': 17, '100101': 17, '001101': 18, '111110': 21, '100011': 22, '110110': 15, '110101': 17, '000010': 15, '001010': 19, '111001': 16, '010010': 17, '111111': 16, '011100': 14, '110010': 10, '000111': 20, '101001': 16, '101111': 19, '111010': 15, '110011': 17, '000101': 9, '001001': 16, '010011': 12, '101000

### e. Run post-processing at once

Since `WavesExecuter` does not provide any post-processing feature.
Neither `.analyze()` nor `multiAnalysis` will provide any useful information.

In [8]:
experiment_workflow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
)

| 0/3 - Analysis:  - 00:00 < ?

| ultimate_question: 
| ultimate_question: 
| ultimate_question: 
| "report.001" has been completed.


'6091db19-b385-4b74-b18c-3d551dd8e6b1'

### e. Read exported multimanager data


In [9]:
multi_exps1_reades = experiment_workflow.multiRead(
    save_location=".",
    summoner_name="qurrium.waves_executer.001",
)

| Retrieve qurrium.waves_executer.001...
| at: qurrium.waves_executer.001


| 0/3   0%|          | - Loading 3 experiments ... - 00:00 < ?

---

### Post-Process Availablities and Version Info

In [10]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No    
 -